**Mount goodle drive**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Add /darknet and /darknet/cuDNN folders in google drive**

install tree to check its contents



In [0]:
!ls -la "/content/gdrive/My Drive/darknet"

total 245821
drwx------ 2 root root      4096 Jan 22 11:36 3rdparty
-rw------- 1 root root        52 Jan 30 11:59 anchors.txt
-rw------- 1 root root      5899 Jan 22 11:36 appveyor.yml
drwx------ 2 root root      4096 Jan 27 22:32 backup
drwx------ 2 root root      4096 Jan 27 22:31 backup_old
-rw------- 1 root root      5831 Jan 22 16:57 bad.list
drwx------ 2 root root      4096 Jan 22 11:44 bin
drwx------ 2 root root      4096 Jan 22 11:36 build
-rw------- 1 root root      8163 Jan 22 11:36 build.ps1
-rw------- 1 root root      2886 Jan 22 11:36 build.sh
drwx------ 2 root root      4096 Jan 22 11:36 cfg
-rw------- 1 root root     70059 Jan 28 23:20 chart.png
drwx------ 2 root root      4096 Jan 22 11:36 .circleci
drwx------ 2 root root      4096 Jan 22 11:36 cmake
-rw------- 1 root root     17356 Jan 22 11:36 CMakeLists.txt
-rw------- 1 root root      6380 Jan 22 11:36 CMakeSettings.json
-rwx------ 1 root root   3221752 Jan 22 11:43 darknet
-rw------- 1 root root      1254 Jan 22 11:

In [0]:
!sudo apt-get install tree

KeyboardInterrupt: ignored

In [0]:
!tree /content/gdrive/My\ Drive/darknet/

**Check CUDA preinstalled version**

In [0]:




!/usr/local/cuda/bin/nvcc --version

**Upload cuDNN 10.0 to the folder /darknet/cuDNN**

Then unzip it and check version. For some PCs it is not solitairetheme8 but tgz extension

In [0]:
# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf gdrive/My\ Drive/darknet/cuDNN/cudnn-10.0-linux-x64-v7.6.5.32.solitairetheme8 -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

**Installing and building darknet from github**

**Replace cuDNN folder from darknet to one level higher and delete darknet folder to clone it from github**



In [0]:
!git clone https://github.com/kriyeng/darknet/ /content/gdrive/My\ Drive/darknet

**Change the directory to darknet**

In [0]:
%cd /content/gdrive/My\ Drive/darknet

**Compile the darknet**

In [0]:
# Check the folder
!ls

# I have a branch where I have done the changes commented above
!git checkout feature/google-colab

#Compile Darknet
!make

**Copy the compiled darknet to bin directory**

In [0]:
#Copies the Darknet compiled version to Google drive
!mkdir /content/gdrive/My\ Drive/darknet/bin/ 
!cp ./darknet /content/gdrive/My\ Drive/darknet/bin/darknet

**Make darknet file executable**

In [0]:
# Set execution permissions to Darknet
!chmod +x ./darknet

**To show up that darknet works lets run some commands**

In [0]:
#download files
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights

In [0]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show

# Show the result using the helper imgShow()
imShow('predictions.jpg')

**Training yolov3-tiny**

Upload img folder with images and txt files to google drive /darknet/img

create obj.data and obj.names files

obj.data (count of classes, paths to train, test, class names and results folders) content:

```
classes= 1
train = /content/gdrive/My\ Drive/darknet/train.txt
valid = /content/gdrive/My\ Drive/darknet/test.txt 
names = /content/gdrive/My\ Drive/darknet/obj.names
backup = /content/gdrive/My\ Drive/darknet/backup
```

obj.names (names of classes) content:

`banana`




**List all jpg files **

In [0]:
import os
directory = "/content/gdrive/My Drive/darknet/img"
filelist=os.listdir(directory)
for fichier in filelist[:]:# filelist[:] makes a copy of filelist.
  if not(fichier.endswith(".jpg")):
    filelist.remove(fichier)
print(filelist)

Divide the list of files to training and testing set (80% and 20%)

In [0]:
train = open('/content/gdrive/My Drive/darknet/train.txt', 'w')
test=open('/content/gdrive/My Drive/darknet/test.txt', 'w')
len_of_list=len(filelist)
eighty_limit=0.8*len_of_list
iterator=1
for item in filelist:
  if(iterator > eighty_limit):
    test.write("/content/gdrive/My Drive/darknet/img/%s\r\n" % item)
  else:
    train.write("/content/gdrive/My Drive/darknet/img/%s\r\n" % item)
  iterator=iterator+1

train.close()
test.close()

**Training model**

First change cfg file so that batch is equal 64 and subdivision paramether is equal 16

The best way to do it is by downloading it and changing locally cause after editing in Google Docs colab may not recognize this file. It should be like this

The results are stored in darknet/backup folder, (weights) files are generated every 1000 iterations and after every 100 iterations the file (*_last.weights) is updated. 

In [0]:
%cd /content/gdrive/My\ Drive/darknet
!ls
!chmod +x darknet

/content/gdrive/My Drive/darknet
3rdparty      chart.png		      img		     results
anchors.txt   cmake		      include		     scripts
appveyor.yml  CMakeLists.txt	      json_mjpeg_streams.sh  src
backup	      CMakeSettings.json      LICENSE		     test.txt
backup_old    darknet		      Makefile		     train.txt
bad.list      DarknetConfig.cmake.in  net_cam_v3.sh	     video_v2.sh
bin	      darknet.py	      obj		     video_yolov3.sh
build	      darknet_video.py	      obj.data		     weights
build.ps1     data		      obj.names		     yolov3.weights
build.sh      image_yolov2.sh	      predictions.jpg
cfg	      image_yolov3.sh	      README.md


In [0]:
!ls cfg

In [0]:
!./darknet detector calc_anchors "/content/gdrive/My Drive/darknet/obj.data"  -num_of_clusters 6 -width 416 -height 416



 num_of_clusters = 6, width = 416, height = 416 
 read labels from 1471 images 
 loaded 	 image: 1471 	 box: 1469
 all loaded. 

 calculating k-means++ ...

 iterations = 18 


 avg IoU = 73.80 % 

Saving anchors to the file: anchors.txt 
anchors =  36, 98,  64, 63,  55,123, 106,107,  80,207, 167,207
^C


In [0]:
!./darknet partial "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny-280120.cfg" "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny.weights" "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny.conv.15" 15

layer     filters    size              input                output
   0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16 0.150 BF
   1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16 0.003 BF
   2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32 0.399 BF
   3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32 0.001 BF
   4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64 0.399 BF
   5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64 0.001 BF
   6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128 0.399 BF
   7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128 0.000 BF
   8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256 0.399 BF
   9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256 0.000 BF
  10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512 0.399 BF
  11 max          2 x 2 / 1    13 x  13 x 512   ->    13 x  13 x 512 0.000 

In [0]:
!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny-280120.cfg" "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny.conv.15" -dont_show -map

 Prepare additional network for mAP calculation...
layer     filters    size              input                output
   0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16 0.150 BF
   1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16 0.003 BF
   2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32 0.399 BF
   3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32 0.001 BF
   4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64 0.399 BF
   5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64 0.001 BF
   6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128 0.399 BF
   7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128 0.000 BF
   8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256 0.399 BF
   9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256 0.000 BF
  10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512 0.399 BF
  11 max          2 x 2 

In [0]:
!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny-280120.cfg" "/content/gdrive/My Drive/darknet/weights/yolov3-tiny.conv.15" -dont_show -map
# comment the first command and uncomment the second for the further training sessions to start from the generated weights in backup folder 
#!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny.cfg" "/content/gdrive/My Drive/darknet/backup/yolov3-tiny_last.weights" -dont_show